In [3]:
from bs4 import BeautifulSoup
import requests
import re
import time
import pandas as pd

cookies = {}
raw_cookies=input("Enter the cookies:")
for line in raw_cookies.split(';'):
    key,value=line.split('=',1)#1代表只分一次，得到两个数据
    cookies[key[1:]]=value

df = pd.read_csv('user_list.csv')
def get_info(soup): 
    a = str(soup)
    pattern = re.compile('共同的喜好\((\d+)',re.S)
    same_num = (re.findall(pattern,a))   
    if len(same_num)>0:
        same_like = int(same_num[0])
    else:
        same_like = 0           
    
    b = soup.find_all("h2")
    text = ""
    for i in b:
        if "电影" in str(i):
            text = str(i)
    pattern2 = re.compile('target.+>(\d+\S+)部看过',re.S)
    movie_num = re.findall(pattern2,text)
    
    if len(movie_num)>0:
        num_movies = int(movie_num[0])
    else:
        num_movies = 0
    
    if same_like > 0 and num_movies > 0 :
        return same_like,num_movies,same_like/num_movies
    else:
        return 0,0,0        

df_user = pd.DataFrame(columns=['url', 'samelike', 'watched', 'rate_self'])

for user in df["User"]:
    print(user)
    r= requests.get(user,cookies=cookies)
    soup = BeautifulSoup(r.content,"html.parser")
    same_like,watched,rate_self = get_info(soup)
    
    df_user.loc[-1] = [user,same_like,watched,rate_self]  # adding a row
    df_user.index = df_user.index + 1  # shifting index
    df_user = df_user.sort_index()  # sorting by index  
    time.sleep(2)
    df_user.to_csv('user_rate.csv', encoding='utf-8')

Enter the cookies:bid=NGGgsiyNckM; __yadk_uid=RktyRu5hlNGGUEalCjxNB5RjyZmB52gT; _ga=GA1.2.927319413.1499268091; ps=y; loc-last-index-location-id="108296"; ll="108296"; _vwo_uuid_v2=1FB849B03DE973D2453AADA1D8D9712B|bc16804a8d80979bacd49595dea6e7c1; gr_user_id=21a1885a-4143-4230-8a64-90ac57969895; _pk_ref.100001.8cb4=%5B%22%22%2C%22%22%2C1499597244%2C%22https%3A%2F%2Fwww.baidu.com%2Flink%3Furl%3DffSue4c5h_Wu8FYSn9jqQoq73yIpyfcKejontW0Nk-RJCGHCXpOKkeqk6NEoe7Ig%26wd%3D%26eqid%3Db4972d730003a9540000000659610686%22%5D; push_doumail_num=0; __utmt=1; push_noty_num=0; _pk_id.100001.8cb4=d1a83588c38b78cf.1499268088.9.1499597425.1499530896.; _pk_ses.100001.8cb4=*; __utma=30149280.927319413.1499268091.1499530898.1499597246.7; __utmb=30149280.3.10.1499597246; __utmc=30149280; __utmz=30149280.1499530898.6.2.utmcsr=baidu|utmccn=(organic)|utmcmd=organic; __utmv=30149280.3906; ap=1; ue="yuchijing@126.com"; dbcl2="39068167:ji6JDR1UMZk"
https://www.douban.com/people/richer725
https://www.douban.com/peopl

ConnectionError: HTTPSConnectionPool(host='www.douban.com', port=443): Max retries exceeded with url: /people/113341230 (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x0000026894DFE550>: Failed to establish a new connection: [WinError 10060] 由于连接方在一段时间后没有正确答复或连接的主机没有反应，连接尝试失败。',))